In [100]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [118]:
N_EPOCHS = 100
N_TRIALS = 3
BATCH_SIZE = 256

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [119]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

train_size = x_train.shape[0]

In [117]:
noise = 30
x_train = np.clip(x_train + np.random.normal(0, noise, x_train.shape), 0, 255) / 255
x_val = np.clip(x_val + np.random.normal(0, noise, x_val.shape), 0, 255) / 255
x_test = np.clip(x_test + np.random.normal(0, noise, x_test.shape), 0, 255) / 255

In [112]:
results = {}

# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]

In [113]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [114]:
for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=10, **models_hyperparameters[test_model])
        
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(2 * (i + 1)/ (N_EPOCHS / 2)) * train_size)
            samples_ids = np.random.choice(range(train_size), n_samples, replace=False)

            model.fit(
                x_train[samples_ids], y_train[samples_ids], 
                validation_data=(x_val, y_val),
                epochs=1, batch_size=BATCH_SIZE,
                verbose=1, callbacks=[early_stopping]
            )
        
        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
        
        model_scores.append(accuracy)

    results[test_model] = model_scores

test_model_1


  0%|          | 0/3 [00:00<?, ?it/s]

6/6 [==============================] - 0s 15ms/step - loss: 1.3038 - accuracy: 0.5280


 33%|███▎      | 1/3 [02:19<04:38, 139.04s/it]

6/6 [==============================] - 0s 13ms/step - loss: 1.3171 - accuracy: 0.5200


 67%|██████▋   | 2/3 [04:30<02:14, 134.30s/it]

6/6 [==============================] - 0s 8ms/step - loss: 1.3533 - accuracy: 0.5047


100%|██████████| 3/3 [06:36<00:00, 132.17s/it]


In [116]:
results

{'test_model_1': [0.527999997138977, 0.5199999809265137, 0.5046666860580444]}

In [ ]:
{'test_model_1': [0.5246666669845581, 0.4620000123977661, 0.4846666753292084]}

In [97]:
(np.tanh(2 * (20 + 1)/ (100 / 2)))

0.6858090622290945